<a href="https://colab.research.google.com/github/debsk/DataSicence/blob/master/cxodiugo_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Projeto cxodiugo - Google Colab
Este notebook adapta todo o projeto `cxodiugo_projeto` para o ambiente do Google Colab, com suporte a TensorFlow, análise de dados e modelo LSTM com Attention.

👉 Basta rodar célula por célula, sem instalar nada localmente!

In [5]:
# ✅ Instalação de dependências no Colab
!pip install pandas numpy matplotlib seaborn tensorflow

## 📥 Simulação da coleta de dados MT5
Como o MetaTrader 5 não roda diretamente no Colab, aqui simulamos os dados como se fossem importados de lá.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Carregar os dados reais do arquivo CSV
df = pd.read_csv("XAUUSD_H1_10anos.csv")

# Verifica se há uma coluna de data e converte, se necessário
# Substitua 'time' pelo nome da coluna de data/hora, se for diferente
if not np.issubdtype(df['time'].dtype, np.datetime64):
    df['time'] = pd.to_datetime(df['time'])

# Garantir que os dados estejam ordenados por tempo
df = df.sort_values('time').reset_index(drop=True)

# Recalcular a coluna 'close', se quiser manter a lógica do dado simulado
# (opcional — comente se não quiser modificar o close real)
# df['close'] = df[['open', 'high', 'low']].mean(axis=1) + np.random.normal(0, 1, len(df))

# Exibir primeiras linhas para conferência
print("Dados carregados do CSV:")
df.head()


Dados carregados do CSV:


,time,open,high,low,close,tick_volume,spread,real_volume,ATR,VWAP,SuperTrend,Fractal_High,Fractal_Low,Hurst_Exponent
0,2014-01-02 09:00:00,1225.12,1228.51,1220.59,1222.90,4162,42,2258940,7.92,1222.900000,NaN,NaN,NaN,NaN
1,2014-01-02 10:00:00,1222.93,1224.00,1219.78,1221.74,3129,44,1977417,4.22,1222.402175,NaN,NaN,1219.78,NaN
2,2014-01-02 11:00:00,1221.89,1221.99,1219.18,1221.32,2752,35,2504400,2.81,1222.105636,NaN,NaN,1219.18,NaN
3,2014-01-02 12:00:00,1221.33,1221.80,1219.22,1221.22,2153,38,1951500,2.58,1221.949292,NaN,NaN,NaN,NaN
4,2014-01-02 13:00:00,1221.21,1221.47,1216.43,1218.02,4022,41,2782348,5.04,1220.974843,NaN,NaN,1216.43,NaN


## 📐 Cálculo de Indicadores Técnicos
ATR, VWAP, SuperTrend, Fractais e Hurst Exponent.

In [9]:
# ATR
df['ATR'] = df['high'] - df['low']
# VWAP
df['VWAP'] = (df['close'] * df['tick_volume']).cumsum() / df['tick_volume'].cumsum()
# SuperTrend (simples - média móvel)
df['SuperTrend'] = df['close'].rolling(window=10).mean()
# Fractais
df['Fractal_High'] = df['high'][(df['high'] > df['high'].shift(1)) & (df['high'] > df['high'].shift(-1))]
df['Fractal_Low'] = df['low'][(df['low'] < df['low'].shift(1)) & (df['low'] < df['low'].shift(-1))]
# Hurst Exponent
def hurst(ts):
    lags = range(2, 20)
    tau = [np.std(ts[lag:] - ts[:-lag]) for lag in lags]
    hurst_exp = np.polyfit(np.log(lags), np.log(tau), 1)[0]
    return hurst_exp
df['Hurst'] = df['close'].rolling(window=500).apply(hurst, raw=True)
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume,ATR,VWAP,SuperTrend,Fractal_High,Fractal_Low,Hurst_Exponent,Hurst
66292,2025-04-04 19:00:00,3021.89,3036.63,3017.68,3030.93,5061,12,0,18.95,1657.364404,3071.238,NaN,NaN,0.375252,0.375252
66293,2025-04-04 20:00:00,3030.96,3032.79,3015.78,3025.82,4938,10,0,17.01,1657.386696,3064.531,NaN,3015.78,0.382113,0.382113
66294,2025-04-04 21:00:00,3025.80,3027.76,3017.94,3025.33,4432,6,0,9.82,1657.406696,3058.646,NaN,NaN,0.391197,0.391197
66295,2025-04-04 22:00:00,3025.50,3040.79,3022.01,3036.35,4610,5,0,18.78,1657.427666,3053.214,NaN,NaN,0.398577,0.398577
66296,2025-04-04 23:00:00,3036.30,3040.87,3033.71,3037.34,2603,17,0,7.16,1657.439515,3045.919,NaN,NaN,0.405728,0.405728


## 🤖 Modelo LSTM com Attention
Aqui usamos TensorFlow para criar uma LSTM com atenção para prever o fechamento do XAUUSD.

In [10]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.optimizers import Adam

# Preprocessamento
scaler = MinMaxScaler()
scaled_close = scaler.fit_transform(df[['close']].dropna())

# Criar janelas de sequência
X, y = [], []
window_size = 50
for i in range(len(scaled_close) - window_size):
    X.append(scaled_close[i:i+window_size])
    y.append(scaled_close[i+window_size])

X, y = np.array(X), np.array(y)

# Arquitetura LSTM com Attention (simples)
input_seq = Input(shape=(window_size, 1))
lstm_out = LSTM(64, return_sequences=True)(input_seq)
attention = Attention()([lstm_out, lstm_out])
context = Concatenate()([lstm_out, attention])
flat = Dense(32, activation='relu')(context[:, -1])
output = Dense(1)(flat)
model = Model(inputs=input_seq, outputs=output)

model.compile(optimizer=Adam(0.001), loss='mse')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 50, 64)    │     16,896 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 50, 64)    │          0 │ lstm[0][0],       │
│ (Attention)         │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 50, 128)   │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 128)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      4,128 │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         33 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,057 (82.25 KB)

 Trainable params: 21,057 (82.25 KB)

 Non-trainable params: 0 (0.00 B)

### ⏱️ Treinamento do modelo

In [11]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 37s 21ms/step - loss: 9.3959e-04 - val_loss: 1.1499e-04
Epoch 2/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 39s 20ms/step - loss: 1.0842e-05 - val_loss: 2.9240e-05
Epoch 3/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - loss: 6.8335e-06 - val_loss: 3.7725e-05
Epoch 4/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 40s 19ms/step - loss: 7.8506e-06 - val_loss: 5.1558e-05
Epoch 5/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 44s 21ms/step - loss: 5.2534e-06 - val_loss: 3.7450e-05
Epoch 6/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 39s 20ms/step - loss: 4.8252e-06 - val_loss: 2.2145e-05
Epoch 7/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 3.6241e-06 - val_loss: 4.5349e-05
Epoch 8/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 3.9355e-06 - val_loss: 7.6949e-06
Epoch 9/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 3.3340e-06 - val_loss: 2.4392e-05
Epoch 10/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - loss: 2.9151e-06 - val_loss: 2.2355e-05


## 💾 Salvar dados em CSV

In [19]:
df.to_csv("C://Users//Debinha//Downloads//XAUUSD_simulado_com_indicadores.csv", index=False)
print("Arquivo salvo!")

Arquivo salvo!
